In [1]:
import re
import ast
import pyspark
import time
import datetime
import html
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
#visualisation
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.charts import Histogram, Scatter, Bar, TimeSeries, show, \
                         output_file, output_notebook, color, marker, defaults
from bokeh.layouts import column
from bokeh.models import NumeralTickFormatter
import pandas as pd

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [3]:
# Domyślny rozmiar wykresów Bokeh:
defaults.width = 900
defaults.height = 500

In [4]:
# Funkcja pomocnicza: wyszukiwanie podanego atrybutu w linii pliku XML,
# zwraca wartość znalezionego atrybutu lub None jeśli atrybut nie istnieje

def attribute_search(attribute, string):
    result = re.search(attribute + '=\"(.*?)\"', string)
    if result:
        return result.group(1).replace('"', '')
    else:
        return None

In [5]:
# Funkcje pomocnicze: interpretacja plików XML (różne schematy danych)
# ze Stackoverflow

def tags_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['TagName'] = attribute_search('TagName', c)
    count = attribute_search('Count', c)
    row['Count'] = int(count) if count else None
    excerpt = attribute_search('ExcerptPostId', c)
    row['ExcerptPostId'] = int(excerpt) if excerpt else None
    wiki = attribute_search('WikiPostId', c)
    row['WikiPostId'] = int(wiki) if wiki else None
    return pyspark.Row(**row)

def badges_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['UserId'] = int(attribute_search('UserId', c))
    row['Name'] = attribute_search('Name', c)
    row['Date'] = datetime.datetime.strptime(attribute_search('Date', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['Class'] = int(attribute_search('Class', c))
    row['TagBased'] = ast.literal_eval(attribute_search('TagBased', c))
    return pyspark.Row(**row)

def users_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['Reputation'] = int(attribute_search('Reputation', c))
    row['CreationDate'] = datetime.datetime.strptime(attribute_search('CreationDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['DisplayName'] = attribute_search('DisplayName', c)
    row['LastAccessDate'] = datetime.datetime.strptime(attribute_search('LastAccessDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['WebsiteUrl'] = attribute_search('WebsiteUrl', c)
    row['Location'] = attribute_search('Location', c)
    age = attribute_search('Age', c)
    row['Age'] = int(age) if age else None
    row['Views'] = int(attribute_search('Views', c))
    row['UpVotes'] = int(attribute_search('UpVotes', c))
    row['DownVotes'] = int(attribute_search('DownVotes', c))
    return pyspark.Row(**row)

def posts_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['PostTypeId'] = int(attribute_search('PostTypeId', c))
    found_id = attribute_search('ParentId', c)
    row['ParentId'] = int(found_id) if found_id else None
    found_id = attribute_search('AcceptedAnswerId', c)
    row['AcceptedAnswerId'] = int(found_id) if found_id else None
    row['CreationDate'] = datetime.datetime.strptime(attribute_search('CreationDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['Score'] = int(attribute_search('Score', c))
    vc = attribute_search('ViewCount', c)
    row['ViewCount'] = int(vc) if vc else None
    owner = attribute_search('OwnerUserId', c)
    row['OwnerUserId'] = int(owner) if owner else None
    lasted = attribute_search('LastEditorUserId', c)
    row['LastEditorUserId'] = int(lasted) if lasted else None

    row['Body'] = re.sub('(<!--.*?-->|<[^>]*>)', '', html.unescape(attribute_search('Body', c)))
    title = attribute_search('Title', c)
    row['Title'] = title if title else None
    tags = attribute_search('Tags', c)
    row['Tags'] = html.unescape(tags).replace('<', '').replace('>', ' ') if tags else None
    date = attribute_search('ClosedDate', c)
    row['ClosedDate'] = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.%f") if date else None

    count = attribute_search('AnswerCount', c)
    row['AnswerCount'] = int(count) if count else None
    count = attribute_search('CommentCount', c)
    row['CommentCount'] = int(count) if count else None
    count = attribute_search('FavoriteCount', c)
    row['FavoriteCount'] = int(count) if count else None
        
    return pyspark.Row(**row)

def comments_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['PostId'] = int(attribute_search('PostId', c))
    row['Score'] = int(attribute_search('Score', c))
    row['Text'] = re.sub('(<!--.*?-->|<[^>]*>)', '', html.unescape(attribute_search('Text', c)))
    row['CreationDate'] = datetime.datetime.strptime(attribute_search('CreationDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    user = attribute_search('UserId', c)
    row['UserId'] = int(user) if user else None
    return pyspark.Row(**row)

def post_history_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['PostHistoryTypeId'] = int(attribute_search('PostHistoryTypeId', c))
    row['PostId'] = int(attribute_search('PostId', c))
    comm = attribute_search('Comment', c)
    row['Comment'] = comm if comm else None
    text = attribute_search('Text', c)
    row['Text'] = re.sub('(<!--.*?-->|<[^>]*>)', '', html.unescape(text)) if text else None
    return pyspark.Row(**row)

def post_links_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['CreationDate'] = datetime.datetime.strptime(attribute_search('CreationDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['PostId'] = int(attribute_search('PostId', c))
    row['RelatedPostId'] = int(attribute_search('RelatedPostId', c))
    row['LinkTypeId'] = int(attribute_search('LinkTypeId', c))
    return pyspark.Row(**row)

In [6]:
# Wczytanie danych do RDD z plików XML, a następnie konwersja RDD do DF: 
xml_load_path = 'file:///home/marek/Dokumenty/Notebooks/bd/gis_stack_spark/data/'

# Słownik: nazwa pliku i odpowiadająca mu funkcja pomocnicza interpretująca
# schemat danych w pliku XML
xml_load_list = {'Tags.xml': tags_from_xml, 'Badges.xml': badges_from_xml, \
                 'Users.xml': users_from_xml,'Posts.xml': posts_from_xml, \
                 'Comments.xml': comments_from_xml,'PostHistory.xml': post_history_from_xml, \
                 'PostLinks.xml': post_links_from_xml}

tags_rdd = sc.textFile(xml_load_path + 'Tags.xml').filter(lambda line: "row" in line) \
             .map(lambda l: xml_load_list['Tags.xml'](l))

badges_rdd = sc.textFile(xml_load_path + 'Badges.xml').filter(lambda line: "row" in line) \
               .map(lambda l: xml_load_list['Badges.xml'](l))

users_rdd = sc.textFile(xml_load_path + 'Users.xml').filter(lambda line: "row" in line) \
              .map(lambda l: xml_load_list['Users.xml'](l))

posts_rdd = sc.textFile(xml_load_path + 'Posts.xml').filter(lambda line: "row" in line) \
              .map(lambda l: xml_load_list['Posts.xml'](l))

comments_rdd = sc.textFile(xml_load_path + 'Comments.xml').filter(lambda line: "row" in line) \
                 .map(lambda l: xml_load_list['Comments.xml'](l))

post_history_rdd = sc.textFile(xml_load_path + 'PostHistory.xml').filter(lambda line: "row" in line) \
                     .map(lambda l: xml_load_list['PostHistory.xml'](l))

post_links_rdd = sc.textFile(xml_load_path + 'PostLinks.xml').filter(lambda line: "row" in line) \
                   .map(lambda l: xml_load_list['PostLinks.xml'](l))

# Konwersja na DataFrame:
users = sqlContext.createDataFrame(users_rdd)
badges = sqlContext.createDataFrame(badges_rdd)
posts = sqlContext.createDataFrame(posts_rdd)
tags = sqlContext.createDataFrame(tags_rdd)
comments = sqlContext.createDataFrame(comments_rdd)
post_history = sqlContext.createDataFrame(post_history_rdd)
post_links = sqlContext.createDataFrame(post_links_rdd)

In [ ]:
# Liczba użytowników (oś Y) vs liczba postów (oś X):
# DF 'posts' kolumna OwnerUserId = DF 'users' kolumna Id --> można wyjąć np. DisplayName użytkownika
# do samego policzenia: groupby OwnerUserId w tabeli posts

users_by_posts = posts.groupBy('OwnerUserId').count()\
                      .select(col('OwnerUserId').alias('user'), col('count').alias('number_posts'))\
                      .groupBy('number_posts').count()\
                      .select(col('number_posts'), col('count').alias('number_users'))\
                      .orderBy('number_posts')

In [ ]:
# Wersja z RDD:
# x = users_by_posts.rdd.map(lambda x: x.number_posts).collect()
# y = users_by_posts.rdd.map(lambda y: y.number_users).collect()

# lub z pandas DF:
x = users_by_posts.toPandas()['number_posts'].tolist()
y = users_by_posts.toPandas()['number_users'].tolist()

In [ ]:
# Użycie Bokeh do wyświetlenia danych
output_notebook()
plot = figure(plot_width=900, plot_height=500,\
              x_axis_type="log", y_axis_type="log",\
              x_axis_label="Liczba postów", y_axis_label="Liczba użytkowników")
plot.circle(x, y, size=6, color="green", alpha=0.4)
show(plot)

In [ ]:
# Aktywność community GIS: Number of Questions vs Answers
# posts -- PostTypeId = 1 (Question)
# --> AnswerCount

questions_vs_answers = posts.select(col('Id'), col('AnswerCount')).where(posts.PostTypeId == 1)\
                            .groupBy('AnswerCount').count()\
                            .select(col('AnswerCount').alias('answers'), col('count').alias('number_questions'))\
                            .orderBy('answers')

In [ ]:
# wersja zamiast .toPandas() -- tworzy listę
#x = questions_vs_answers.rdd.map(lambda x: x.answers).collect()
#top = questions_vs_answers.rdd.map(lambda y: y.number_questions).collect()

In [ ]:
# wykres finalny
output_notebook()
plot = Bar(questions_vs_answers.toPandas(), 'answers', values='number_questions', \
           title="Odpowiedzi vs liczba pytań", legend='top_right', color="navy", \
           ylabel='Pytania', xlabel='Odpowiedzi')

show(plot)

In [ ]:
total_questions = posts.select(posts.Id).where(posts.PostTypeId == 1).count()

unanswered_questions = questions_vs_answers.select(col('number_questions')) \
                       .where(questions_vs_answers.answers == 0) \
                       .collect()[0].number_questions

In [ ]:
print("Liczba pytań na forum GIS:    {}".format(total_questions))
print("Liczba pytań bez odpowiedzi:  {}".format(unanswered_questions))
print("Procent pytań bez odpowiedzi: {:.2%}".format(unanswered_questions/total_questions))

In [ ]:
posts.printSchema()

In [ ]:
tags.printSchema()

In [7]:
# Najpopularniejsze tagi na forum
popular_tags = tags.select(tags.TagName, tags.Count)\
                   .sort(tags.Count, ascending=False).toPandas()

In [ ]:
popular_tags.head(40)

In [8]:
output_notebook()
plot = Bar(popular_tags.head(20), 'TagName', values='Count', \
           title="Najpopularniejsze tagi na forum GIS", legend=False, color="darkviolet", \
           ylabel='Liczba postów', xlabel='Tag (nazwa)')
show(plot)

Loading BokehJS ...

In [ ]:
# Wstępna weryfikacja danych / zapytań

In [ ]:
posts.filter(posts.Tags.rlike(r'\bpostgis\b')).count()

In [ ]:
posts.count()

In [ ]:
# Zapytanie dla jednego produktu GIS (tu tag 'arcgis-desktop'):
gis_posts_data = posts.filter(posts.Tags.rlike(r'\barcgis-desktop\b')) \
    .select(concat_ws('.', year("CreationDate"), \
                           lpad(month("CreationDate"), 2, "0")).alias('Months')) \
    .groupBy('Months').count().sort('Months').toPandas()

gis_posts_data.rename(columns={'count': 'ArcGIS'}, inplace=True)

# Można dodać kolejną kolumnę z danymi do DF:
# arcgis_desktop['PostGIS'] = postgis['PostGIS']

In [ ]:
# Dla kilku produktów: iterujemy po liście tagów i składamy kolejne kolumny w DF pandas

In [ ]:
gis_products = ['qgis', 'arcgis-desktop', 'postgis', 'openstreetmap', 'grass', 'google-maps']
# gis_percentages = ['%{0}'.format(i) for i in gis_products]

In [ ]:
# Analiza dla produktów ArcGIS (różne wersje w tagach)
gis_products = ['arcgis-desktop', 'arcgis-server', 'arcgis-10.0', \
                'arcgis-10.1', 'arcgis-10.2', 'arcgis-10.3' ,'arcgis-javascript-api']

In [ ]:
# Wyliczamy totals (dla wszystkich postow) w gis_posts_data.
# Dzięki temu mamy DF do którego potem doklejamy kolumny
# z wynikami dla innych tagów.
print('Wyliczam statystyki dla wszystkich postow.')
gis_posts_data = posts.filter(posts.Tags.isNotNull()).select(concat_ws('.', year("CreationDate"), \
                              lpad(month("CreationDate"), 2, "0")).alias('Months')) \
                      .groupBy('Months').count().sort('Months').toPandas()

gis_posts_data.rename(columns={'count': 'wszystkie'}, inplace=True)

for gis_p in gis_products:
    print('Wyliczam posty dla produktu: {0}'.format(gis_p))
    product_posts = posts.filter(posts.Tags.isNotNull() & posts.Tags.rlike(r'\b{0}\b'.format(gis_p))) \
        .select(concat_ws('.', year("CreationDate"), lpad(month("CreationDate"), 2, "0")).alias('Months')) \
        .groupBy('Months').count().sort('Months').toPandas()
    product_posts.rename(columns={'count': gis_p}, inplace=True)
    gis_posts_data = pd.merge(gis_posts_data, product_posts, on='Months', how='outer')

print('Gotowe.')

In [ ]:
product_posts

In [ ]:
gis_posts_data

In [ ]:
# Test funkcji .rolling:
gis_posts_data.rolling(on='Months', window=5).mean()

In [ ]:
# Obliczanie % postow dla produktu ws totals
# np. dla produktu qgis obliczamy qgis_p jako
# (qgis / wszystkie) * 100%
for gis_p in gis_products:
    gis_posts_data['%' + gis_p] = (gis_posts_data[gis_p].fillna(0) \
                                   / gis_posts_data.wszystkie) * 100

In [ ]:
gis_posts_data

In [ ]:
# Trend czasowy -- posty na miesiąc
output_notebook()

#tsline = TimeSeries(gis_posts_data, x='Months', y=['wszystkie'] + gis_products, \
#         title="Posty na miesiąc", xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)
tsline = TimeSeries(gis_posts_data, x='Months', y=gis_products, \
         title="Posty na miesiąc (liczba)", xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)

show(column(tsline))

In [ ]:
# Trend czasowy wygładzony -- posty na miesiąc
output_notebook()

#tsline = TimeSeries(gis_posts_data, x='Months', y=['wszystkie'] + gis_products, \
#         title="Posty na miesiąc", xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)
tsline = TimeSeries(gis_posts_data.rolling(on='Months', window=4).mean(), x='Months', y=gis_products, \
         title="Posty na miesiąc (liczba), wygładzone", \
         xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)

show(column(tsline))

In [ ]:
# Trend czasowy -- posty na miesiąc w % do całości
output_notebook()

tsline = TimeSeries(gis_posts_data, x='Months', y=['%{0}'.format(i) for i in gis_products], \
         title="Posty na miesiąc (%całosci)", xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)
#tsline.xaxis.minor_tick_in = -3
#tsline.xaxis.minor_tick_out = 2
#tsline.xaxis.major_tick_out = 3

#tsline.xgrid.grid_line_color = None
tsline.ygrid.grid_line_dash = [6, 4]

show(column(tsline))

In [ ]:
# Ja, jako użytkownik forum, chcę wiedzieć, jak długo zajmie uzyskanie
# dobrej odpowiedzi dla danego produktu (lista produktów).
#
# ile czasu upływa od zadania pytania
# do uzyskania zaakceptowanej odpowiedzi (dla tagów / produktów typu ArcGIS -- wykres??).

In [ ]:
# Wyznaczenie czasu do zaakceptowanej odpowiedzi dla produktu
# najpierw filter
# potem oblicz



In [ ]:
# Czas odpowiedzi
# posts -- PostTypeId = 1 (Question), 2 (Answer)

questions = posts.select(posts.Id, posts.AcceptedAnswerId,\
                         posts.CreationDate.alias('QuestionDate'),\
                         posts.Tags).where(posts.PostTypeId == 1)

# alias('Parent') to obejście problemu
# org.apache.spark.sql.AnalysisException:
# resolved attribute(s) ID missing from ID in operator !Join
# https://issues.apache.org/jira/browse/SPARK-10925
answers = posts.select(posts.Id.alias('AnswerId'), posts.ParentId.alias('Parent'),\
                       posts.CreationDate.alias('AnswerDate')).where(posts.PostTypeId == 2)

In [ ]:
questions.printSchema()
answers.printSchema()

In [ ]:
# UDF do obliczenia różnicy czasu (w sekundach)
# x, y -- datetime
def time_delta(x, y):
    delta = int((y - x).total_seconds())
    return delta

time_delta_udf = udf(time_delta, returnType = IntegerType())

In [ ]:
# join jest dla wszystkich odpowiedzi (a nie tylko AcceptedAnswer)
joined = questions.join(answers, questions.Id == answers.Parent)
#         .select(questions.Id, questions.AcceptedAnswerId, questions.CreationDate, answers.CreationDate)

# dla zaakceptowanych trzeba użyć filtr AcceptedAnswerId.isNotNull()
#df = questions.join(answers, questions.Id == answers.Parent).filter(col('AcceptedAnswerId').isNotNull()).count()

In [ ]:
# Posty "merged" mają często zamienione oryginalne pytanie z późniejszym,
# co sprawia, że czas odpowiedzi jest przed czasem zadania pytania.
# Przykład:
# https://gis.stackexchange.com/questions/124794/installing-qgis-with-ecw-support-on-mac
# w dumpie z 14 marca ma 2 dodatkowe linki (merge)
# trzeba znaleźć posty, które w post_history mają PostHistoryTypeId == 37 lub == 38

In [ ]:
post_history.printSchema()

In [ ]:
# Liczba merged_posts źródłowych i docelowych.
# Posty mające w historii (tabela post_history):
# PostHistoryTypeId
# 37. Post merge source
# 38. Post merge destination
post_history.where((post_history.PostHistoryTypeId == 37) | (post_history.PostHistoryTypeId == 38)).count()

In [ ]:
# (Za dużo otwartych plików)
# questions.join(post_history, questions.Id == post_history.PostId).count()

In [ ]:
# (Za dużo otwartych plików)
# posts.join(post_history, posts.Id == post_history.PostId).first()

#                            .filter(col('AcceptedAnswerId').isNotNull())\


In [ ]:
# ze zbioru postów należy usunąć posty: zmerdżowane (merged)
# oraz duplikaty (duplicate). Merged mają często ujemne czasy
# odpowiedzi, duplikaty -- bardzo długie czasy (jeśli np. nowy
# post jest duplikatem starego postu)
merged_posts_ids = post_history.select(post_history.PostId) \
                   .where((post_history.PostHistoryTypeId == 37) | \
                          (post_history.PostHistoryTypeId == 38))

duplicate_posts_ids = post_links.select(post_links.PostId).where(post_links.LinkTypeId == 3)

remove_ids = merged_posts_ids.union(duplicate_posts_ids).distinct()

In [ ]:
remove_ids.count()

In [ ]:
posts.count()

In [ ]:
#  anti_join using a dataframe:
#    tbl1.as("a").join(tbl2.as("b"), $"a.id" === $"b.id", "left_anti")
# This PR provides serves as the basis for implementing `NOT EXISTS` and `NOT IN 
# (...)` correlated sub-queries. It would also serve as good basis for 
# implementing an more efficient `EXCEPT` operator.
posts_nd = posts.join(remove_ids, posts.Id == remove_ids.PostId, 'left_anti')

In [ ]:
posts_nd.count()

In [ ]:
questions = posts_nd.select(posts_nd.Id, posts_nd.AcceptedAnswerId,\
                         posts_nd.CreationDate.alias('QuestionDate'),\
                         posts_nd.Tags).where(posts_nd.PostTypeId == 1)

In [ ]:
questions.count()

In [ ]:
questions.count()

In [ ]:
answers.count()

In [ ]:
merged_posts_ids.count()

In [ ]:
#  anti_join using a dataframe:
#    tbl1.as("a").join(tbl2.as("b"), $"a.id" === $"b.id", "left_anti")
# This PR provides serves as the basis for implementing `NOT EXISTS` and `NOT IN 
# (...)` correlated sub-queries. It would also serve as good basis for 
# implementing an more efficient `EXCEPT` operator.

questions_nm = questions.join(merged_posts_ids, questions.Id == merged_posts_ids.PostId, 'left_anti')

In [ ]:
answers_nm = answers.join(merged_posts_ids, answers.AnswerId == merged_posts_ids.PostId, 'left_anti')

In [ ]:
questions_nm.count()

In [ ]:
# prosty test po joinie. juz nie mamy df joined (do usuniecia) -- chodzi o pokazanie zasady
# joined_timedelta = joined.withColumn('TimeDelta', time_delta_udf(joined.QuestionDate, joined.AnswerDate))

In [ ]:
# tylko z zaakceptowanymi odpowiedziami (ale uwzględnia pozostałe odpowiedzi dla pytania)
joined_timedelta = \
    questions_nm.join(answers_nm, questions_nm.Id == answers_nm.Parent) \
    .filter(col('AcceptedAnswerId').isNotNull()) \
    .withColumn('TimeDelta', time_delta_udf(questions_nm.QuestionDate, answers_nm.AnswerDate))

In [ ]:
# tylko dla zaakceptowanych odpowiedzi
joined_timedelta = \
    questions_nm.join(answers_nm, questions_nm.Id == answers_nm.Parent) \
    .filter(col('AcceptedAnswerId').isNotNull() & (col('AcceptedAnswerId') == col('AnswerId'))) \
    .withColumn('TimeDelta', time_delta_udf(questions_nm.QuestionDate, answers_nm.AnswerDate))

In [ ]:
# wszystkie z odpowiedziami (nie tylko zaakceptowanymi)
joined_timedelta = questions_nm.join(answers_nm, questions_nm.Id == answers_nm.Parent) \
                               .withColumn('TimeDelta', time_delta_udf(questions_nm.QuestionDate, answers_nm.AnswerDate))

In [ ]:
joined_timedelta.printSchema()

In [ ]:
joined_timedelta.count()

In [ ]:
joined_timedelta.where(joined_timedelta.TimeDelta > 0).count()

In [ ]:
joined_timedelta.collect()

In [ ]:
quantiles = [0.0, 0.25, 0.5, 0.75, 1.0]
qlabels = ["q"+str(int(q * 100)) for q in quantiles]

In [ ]:
joined_timedelta.approxQuantile('TimeDelta', quantiles, 0)

In [ ]:
joined_timedelta.where(joined_timedelta.TimeDelta > 0).approxQuantile('TimeDelta', quantiles, 0)

In [ ]:
joined_timedelta.where((joined_timedelta.TimeDelta > 0) & posts.Tags.rlike(r'\b{0}\b'.format('qgis'))) \
                       .approxQuantile('TimeDelta', quantiles, 0)

In [ ]:
joined_timedelta.where((joined_timedelta.TimeDelta > 0) & posts.Tags.rlike(r'\b{0}\b'.format('qgis')))\
                .sort(joined_timedelta.TimeDelta, ascending=False).collect()

In [ ]:
posts.where(posts.Id == 7078).collect()

In [ ]:
post_history.where(post_history.PostId == 7087).collect()

In [ ]:
# df.groupBy('column').count().rdd.values().histogram()

hist_v = joined_timedelta.groupBy(joined_timedelta.TimeDelta).count().rdd.values().histogram(40)

In [ ]:
count_timedelta = joined_timedelta.groupBy(joined_timedelta.TimeDelta).count() \
                         .select(col('TimeDelta'), col('count').alias('Count'))

In [ ]:
count_timedelta.toPandas()

In [ ]:
hist_v.printSchema()

In [ ]:
hist_v.where(hist_v.Count > 1).sort('Count', ascending=False).collect()

In [ ]:
hist_df = pd.DataFrame(list(zip(list(hist_v)[0], \
                       list(hist_v)[1])), \
                       columns=['bin','frequency'])

In [ ]:
hist_df

In [ ]:
output_notebook()

plot = Bar(hist_df, 'bin', values='frequency', \
           title="Histogram", legend=False, color="darkviolet", \
           ylabel='Freq', xlabel='Bin')
show(plot)

In [ ]:
# .min() -- wyszedł problem ze zmergowanymi postami
# różnica czasu jest ujemna (!!! trzeba odfiltrować merged_posts!!!)
joined_timedelta.select(joined_timedelta.TimeDelta).groupBy().mean().collect()

In [ ]:
joined_timedelta.where(joined_timedelta.TimeDelta == 0).count()

In [ ]:
posts.where(posts.Id == 203380).collect()

In [ ]:
post_history.where(post_history.PostId == 203381).collect()

In [ ]:
# wszystkie z filtrem merged_posts_ids

# the `in` operator in pyspark is broken (2.2.0)
# https://issues.apache.org/jira/browse/SPARK-19701
# Funkcjonalność usunięta:
# https://github.com/apache/spark/pull/17160
#
# trzeba użyć:
# http://takwatanabe.me/pyspark/generated/generated/sql.functions.array_contains.html

#questions.filter([x for x in merged_posts_ids]).show()

# The ANTI JOIN – all values from table1 where not in table2 ????????????
# http://blog.montmere.com/2010/12/08/the-anti-join-all-values-from-table1-where-not-in-table2/

#.join(answers, questions.Id == answers.Parent) \
#                            .withColumn('TimeDelta', time_delta_udf(questions.QuestionDate, answers.AnswerDate))

In [ ]:
joined_timedelta.collect()

In [ ]:
joined_timedelta.select(min(joined_timedelta.TimeDelta)).collect()

In [ ]:
joined_timedelta.where(joined_timedelta.TimeDelta == -110165685).collect()

In [ ]:
posts.where(posts.Id == 124794).collect()

In [ ]:

# jesli w historii jest 37, 38, to są to zmerdżowane posty
# 37 źródło, 38 cel, 38 jest starszy?

# do obliczen czasu trzeba usunac duplikaty
# ale org posty nie maja odpowiedzi?


In [ ]:
post_links.where(post_links.RelatedPostId == 124794).collect()

In [ ]:
post_links.where(post_links.PostId == 43830).collect()

In [ ]:
answers.where(answers.Id == 194).collect()

In [ ]:
# Reputation histogram!!!

#To further study the reason behind the difference in motifs
#across forums, we plotted the distribution of users’ reputations
#in figure 7. We see that the proportion of intermediate users is
#higher in discussion based forums than in fact-based forums,
#which explains the higher representation of flat-hierarchy
#interactions (e.g. users of similar expertise level help each
#other out) in the motifs of Movies, UX, and History.

In [ ]:
# Graf kołowy: (bokeh chord)
#The graph we constructed can be succinctly described as
#an accepted answers graph. Figure 3 explains how questions
#and answers in the forum translate to the graph in our analysis.
#A node exists for each user in the forum. For each accepted
#answer, there is an edge from the asker to the answerer.
#We insert an edge into the graph only if the answer is
#accepted, unlike prior analysis on Q&A forums performed
#by other studies where there are edges for both accepted and
#non-accepted answers.